### Kaggle: Avito Demand Prediction Challenge
source: https://www.kaggle.com/c/avito-demand-prediction


Intro: <font color="red"> criar descrição</font>


Team: <font color="red"> criar nome</font>
- Luis Filipe Kopp (lfkopp)
- Lucas Pérez (lucascperez)
- Raphael Reis (raphacoelho)

# Part two - generating bag of words 


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from collections import Counter
from nltk import download
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [2]:
%matplotlib inline
dir = 'c:/_dados/_avito/'
download('punkt')
stemmer = SnowballStemmer("russian")
stop = set(stopwords.words('russian'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lfkop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
def convert(frase):
    frase2 = []
    for word in str(frase).split():
        frase2.append(stemmer.stem(word))
    return ' '.join(frase2)

In [4]:
df = pd.read_pickle(dir + 'df2.pickle')
display(df.head())
display(df.shape)

,activation_date,category_name,city,deal_probability,description,item_id,month,param_1,param_2,param_3,...,user_id,user_type,mean,std,price_normal,time_diff,per_count,date_from,date_to,time_diff2
0,2017-03-28,Товары для детей и игрушки,Екатеринбург,0.12789,"Кокон для сна малыша,пользовались меньше месяц...",b912c3c6a6ad,3,Постельные принадлежности,NaN,NaN,...,e00f8ff2eaf9,Private,4129.398175,1.353662e+05,-0.013774,NaN,NaN,NaT,NaT,NaN
1,2017-03-26,Мебель и интерьер,Самара,0.00000,"Стойка для одежды, под вешалки. С бутика.",2dac0150717d,3,Другое,NaN,NaN,...,39aeb48f0017,Private,32201.494859,3.555820e+06,-0.004106,NaN,NaN,NaT,NaT,NaN
2,2017-03-20,Аудио и видео,Ростов-на-Дону,0.43177,"В хорошем состоянии, домашний кинотеатр с blu ...",ba83aefab5dc,3,"Видео, DVD и Blu-ray плееры",NaN,NaN,...,91e2f88dd6e3,Private,7518.729245,1.979137e+05,-0.008889,NaN,NaN,NaT,NaT,NaN
3,2017-03-25,Товары для детей и игрушки,Набережные Челны,0.80323,Продам кресло от0-25кг,02996f1dd2ea,3,Автомобильные кресла,NaN,NaN,...,bf5cccea572d,Company,4129.398175,1.353662e+05,-0.007126,NaN,NaN,NaT,NaT,NaN
4,2017-03-16,Автомобили,Волгоград,0.20797,Все вопросы по телефону.,7c90be56d2ab,3,С пробегом,ВАЗ (LADA),2110,...,ef50846afc0b,Private,377983.566760,5.233088e+05,-0.312153,NaN,NaN,NaT,NaT,NaN


(2011862, 25)

In [5]:
bag = df[['item_id', 'category_name', 'parent_category_name', 'title', 'description', 'param_1', 'param_2', 'param_3', 'source', 'deal_probability']].fillna(' ')
bag.head()

,item_id,category_name,parent_category_name,title,description,param_1,param_2,param_3,source,deal_probability
0,b912c3c6a6ad,Товары для детей и игрушки,Личные вещи,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",Постельные принадлежности,,,train,0.12789
1,2dac0150717d,Мебель и интерьер,Для дома и дачи,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",Другое,,,train,0
2,ba83aefab5dc,Аудио и видео,Бытовая электроника,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...","Видео, DVD и Blu-ray плееры",,,train,0.43177
3,02996f1dd2ea,Товары для детей и игрушки,Личные вещи,Автокресло,Продам кресло от0-25кг,Автомобильные кресла,,,train,0.80323
4,7c90be56d2ab,Автомобили,Транспорт,"ВАЗ 2110, 2003",Все вопросы по телефону.,С пробегом,ВАЗ (LADA),2110,train,0.20797


In [6]:
df['full_text'] = (df['description'].astype(str)+' '+df['param_1'].astype(str)+' '+df['param_2'].astype(str)+' '+df['param_3'].astype(str)).str.upper().str.replace('NAN','').str.replace('  ',' ').str.replace('  ',' ')

In [7]:
groups = []
for x in df.groupby('parent_category_name')['parent_category_name'].head(1):
    if x not in groups: groups.append(x)
groups

['Личные вещи',
 'Для дома и дачи',
 'Бытовая электроника',
 'Транспорт',
 'Недвижимость',
 'Животные',
 'Хобби и отдых',
 'Услуги',
 'Для бизнеса']

In [11]:
#df2 = df[['item_id', 'parent_category_name', 'full_text']]
#df2.set_index('item_id', inplace=True)
#df2['full_text'] = df2['full_text'].apply(convert)
#df3 = df2.reset_index()
#df3.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-9b4890eab15d>", line 3, in <module>
    df2['full_text'] = df2['full_text'].apply(convert)
  File "C:\Anaconda3\lib\site-packages\pandas\core\series.py", line 2551, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/src/inference.pyx", line 1521, in pandas._libs.lib.map_infer
  File "<ipython-input-3-b07474e5b23b>", line 4, in convert
    frase2.append(stemmer.stem(word))
  File "C:\Anaconda3\lib\site-packages\nltk\stem\snowball.py", line 3577, in stem
    if rv.endswith(suffix):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
A

KeyboardInterrupt: 

In [30]:
#df2.set_index('item_id', inplace=True)
#df2['description'] = df2['description'].apply(convert)
#df3 = df2.reset_index()
#df3.head()

In [ ]:
df2 = df[['item_id', 'parent_category_name', 'description']]
i =0
for group in groups:
    i += 1
    df4 = df2[df2['parent_category_name'] == group]
    with open(dir+'consolidated_'+str(i)+'.txt', 'wb') as f: 
        for idx, row in df4.iterrows():
            idx = row.item_id
            r = str(row.description).split()
            con = Counter(r).items()
            content = ''
            for item in con:
                a,b = item
                content += str(group) + ';' +str(idx) + ';' + stemmer.stem(str(a)) + ';' + str(a) + "\r\n"
            f.write(content.encode('utf-8'))

In [10]:
word_tokenize('кокон для сна малыша,пользова меньш месяца.')

['кокон', 'для', 'сна', 'малыша', ',', 'пользова', 'меньш', 'месяца', '.']